In [191]:
import pandas as pd 

In [192]:
#loading datasets
df_fdi = pd.read_csv("../data/clean/fdi_percent_melt.csv")
df_fsi =  pd.read_csv("../data/clean/fsi_score_melt.csv")
df_bl =  pd.read_csv("../data/clean/complied_start_end.csv")


In [193]:
#clean fdi df


df_fdi = df_fdi.rename(columns={'FDI as percentage of capital': 'FDI %'})

df_fdi.head()

,Unnamed: 0,Country,Date,FDI %
0,0,Anguilla,2008,49.54
1,1,Antigua and Barbuda,2008,30.97
2,2,Aruba,2008,2.25
3,3,Bahamas,2008,50.75
4,4,Bahrain,2008,29.72


In [194]:
#clean fsi df

df_fsi = df_fsi.drop('Unnamed: 0', axis=1)
df_fsi = df_fsi.drop('Country Code', axis=1)
df_fsi = df_fsi.rename(columns={'Financial Secrecy Index': 'FSI Score'})

df_fsi['Date'] = pd.to_datetime(df_fsi['Date'], format='%d-%m-%Y')
df_fsi['Date'] = df_fsi['Date'].dt.year

df_fsi.head()

,Country,Date,FSI Score
0,Aruba,2011,74.20
1,Anguilla,2011,0.00
2,United Arab Emirates,2011,0.00
3,American Samoa,2011,79.27
4,Antigua and Barbuda,2011,0.00


In [195]:
#drop unneccesary countries

values_to_keep = ['Aruba', 'Bahrain', 'Bermuda', 'Cayman Islands', 'Grenada', 'Macau', 'Mongolia', 'Namibia', 'Oman', 'Saint Lucia', 'South Korea', 'Tunisia']
df_fsi = df_fsi[df_fsi['Country'].isin(values_to_keep)]
df_fdi = df_fdi[df_fdi['Country'].isin(values_to_keep)]

In [196]:
#keeping only start of policy
df_bl = df_bl.drop('End', axis=1)

In [197]:
#merge fdi and bl, remove dates lower than start
fdi_bl = pd.merge(df_fdi, df_bl, on=['Country'])
fdi_bl_final = fdi_bl[fdi_bl['Date'] >= fdi_bl['Start']]
fdi_bl_final

,Unnamed: 0,Country,Date,FDI %,Start
91,292,Bahrain,2017,14.18,2017
94,302,Grenada,2017,75.00,2017
95,304,Mongolia,2017,58.24,2017
96,305,Namibia,2017,12.11,2017
98,312,Saint Lucia,2017,22.50,2017
...,...,...,...,...,...
155,496,Mongolia,2023,43.97,2017
156,497,Namibia,2023,132.25,2017
157,498,Oman,2023,17.84,2019
158,504,Saint Lucia,2023,40.60,2017


In [198]:
#merge fsi and bl, remove dates lower than start
fsi_bl = pd.merge(df_fsi, df_bl, on=['Country'])
fsi_bl_final = fsi_bl[fsi_bl['Date'] >= fsi_bl['Start']]

fsi_bl_final.head()


,Country,Date,FSI Score,Start
34,Bahrain,2018,66.050,2017
37,Grenada,2018,0.000,2017
38,South Korea,2018,0.000,2017
39,Macau,2018,0.000,2017
40,Mongolia,2018,54.175,2017


In [199]:
print(fdi_bl_final)

     Unnamed: 0      Country  Date   FDI %  Start
91          292      Bahrain  2017   14.18   2017
94          302      Grenada  2017   75.00   2017
95          304     Mongolia  2017   58.24   2017
96          305      Namibia  2017   12.11   2017
98          312  Saint Lucia  2017   22.50   2017
..          ...          ...   ...     ...    ...
155         496     Mongolia  2023   43.97   2017
156         497      Namibia  2023  132.25   2017
157         498         Oman  2023   17.84   2019
158         504  Saint Lucia  2023   40.60   2017
159         508      Tunisia  2023   11.32   2017

[61 rows x 5 columns]


In [200]:
fdi_bl_final.to_csv('../data/clean/complied_fdi.csv', index=True)

In [201]:
fsi_bl_final.to_csv('../data/clean/complied_fsi.csv', index=True)

In [202]:
# calculate the average change from start to finish FDI

# Sort the DataFrame by 'Country' and 'Date' to ensure chronological order
df_sorted = fdi_bl_final.sort_values(by=['Country', 'Date'])

# Filter to keep only the first and last entry for each 'Country'
first_last_entries = df_sorted.groupby('Country').agg(
    first_entry=('FDI %', 'first'),
    last_entry=('FDI %', 'last')
).reset_index()

# Merge the original DataFrame with the first and last entries
filtered_df = pd.merge(df_sorted, first_last_entries[['Country', 'first_entry', 'last_entry']], 
                       on='Country', how='inner')

# Filter rows where FDI % is either the first or last entry
filtered_df = filtered_df[(filtered_df['FDI %'] == filtered_df['first_entry']) | 
                          (filtered_df['FDI %'] == filtered_df['last_entry'])]

# Drop the helper columns
filtered_df = filtered_df.drop(columns=['first_entry', 'last_entry', 'Start'])
#filtered_df = filtered_df[filtered_df['Country'] != 'Cayman Islands']
filtered_df

,Unnamed: 0,Country,Date,FDI %
0,354,Aruba,2019,-18.91
4,482,Aruba,2023,-24.42
5,292,Bahrain,2017,14.18
11,484,Bahrain,2023,56.67
12,359,Bermuda,2019,0.46
16,487,Bermuda,2023,16.55
17,393,Cayman Islands,2020,2209.00
20,489,Cayman Islands,2023,2408.03
21,302,Grenada,2017,75.00
27,494,Grenada,2023,48.40


In [203]:

def calculate_percentage_change(group):
    if len(group) >= 2:  # Ensure there are at least 2 entries
        first_fdi = group.iloc[0]['FDI %']
        second_fdi = group.iloc[1]['FDI %']
        
        # Calculate the percentage change
        change = ((second_fdi - first_fdi)/second_fdi)*100
        return pd.Series({'Percentage Change in FDI%': change})
    else:
        return pd.Series({'Percentage Change in FDI %': None})  # If there's less than 2 entries, return None

# Apply the function to each group
percentage_change_df_fdi = filtered_df.groupby('Country').apply(calculate_percentage_change)

# Display the result
print(percentage_change_df_fdi)


                Percentage Change in FDI%
Country                                  
Aruba                           22.563473
Bahrain                         74.977942
Bermuda                         97.220544
Cayman Islands                   8.265262
Grenada                        -54.958678
Mongolia                       -32.453946
Namibia                         90.843100
Oman                            57.399103
Saint Lucia                     44.581281
Tunisia                          7.950530


C:\Users\MJ\AppData\Local\Temp\ipykernel_4816\807787173.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentage_change_df_fdi = filtered_df.groupby('Country').apply(calculate_percentage_change)


In [205]:
#adding average and median
 
average_change = percentage_change_df_fdi['Percentage Change in FDI%'].mean()
median_change = percentage_change_df_fdi['Percentage Change in FDI%'].median()

# Create new rows for the average and median
aggregate_row = pd.Series({'Percentage Change in FDI%': average_change}, name='Average')
median_row = pd.Series({'Percentage Change in FDI%': median_change}, name='Median')

# Append the new rows to the DataFrame
percentage_change_df_fdi_final = pd.concat([percentage_change_df_fdi, aggregate_row.to_frame().T, median_row.to_frame().T])

percentage_change_df_fdi_final['Percentage Change in FDI%'] = percentage_change_df_fdi_final['Percentage Change in FDI%'].round(1)

# Display the final DataFrame
print(percentage_change_df_fdi_final)

                Percentage Change in FDI%
Aruba                                22.6
Bahrain                              75.0
Bermuda                              97.2
Cayman Islands                        8.3
Grenada                             -55.0
Mongolia                            -32.5
Namibia                              90.8
Oman                                 57.4
Saint Lucia                          44.6
Tunisia                               8.0
Average                              31.6
Median                               33.6


In [206]:
# calculate the average change from start to finish FSI

# Sort the DataFrame by 'Country' and 'Date' to ensure chronological order
df_sorted = fsi_bl_final.sort_values(by=['Country', 'Date'])

# Filter to keep only the first and last entry for each 'Country'
first_last_entries = df_sorted.groupby('Country').agg(
    first_entry=('FSI Score', 'first'),
    last_entry=('FSI Score', 'last')
).reset_index()

# Merge the original DataFrame with the first and last entries
filtered_df = pd.merge(df_sorted, first_last_entries[['Country', 'first_entry', 'last_entry']], 
                       on='Country', how='inner')

# Filter rows where FDI % is either the first or last entry
filtered_df = filtered_df[(filtered_df['FSI Score'] == filtered_df['first_entry']) | 
                          (filtered_df['FSI Score'] == filtered_df['last_entry'])]

# Drop the helper columns
filtered_df = filtered_df.drop(columns=['first_entry', 'last_entry', 'Start'])

def calculate_percentage_change(group):
    if len(group) >= 2:  # Ensure there are at least 2 entries
        first_fdi = group.iloc[0]['FSI Score']
        second_fdi = group.iloc[1]['FSI Score']
        
        # Calculate the percentage change
        change = ((second_fdi - first_fdi)/second_fdi)*100
        return pd.Series({'Percentage Change in FSI Score': change})
    else:
        return None  # If there are less than 2 entries, return None to exclude the group

# Apply the function to each group
percentage_change_df_nas = filtered_df.groupby('Country').apply(calculate_percentage_change)

# Display the result
percentage_change_df_fsi = percentage_change_df_nas.dropna()

percentage_change_df_fsi.head()

C:\Users\MJ\AppData\Local\Temp\ipykernel_4816\648704630.py:29: RuntimeWarning: invalid value encountered in scalar divide
  change = ((second_fdi - first_fdi)/second_fdi)*100
C:\Users\MJ\AppData\Local\Temp\ipykernel_4816\648704630.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentage_change_df_nas = filtered_df.groupby('Country').apply(calculate_percentage_change)


,Percentage Change in FSI Score
Country,
Aruba,-3.313359
Bahrain,-20.200182
Bermuda,-11.908397
Cayman Islands,1.169211
Mongolia,-2.652771


In [207]:
#adding average and median
 
average_change = percentage_change_df_fsi['Percentage Change in FSI Score'].mean()
median_change = percentage_change_df_fsi['Percentage Change in FSI Score'].median()

# Create new rows for the average and median
aggregate_row = pd.Series({'Percentage Change in FSI Score': average_change}, name='Average')
median_row = pd.Series({'Percentage Change in FSI Score': median_change}, name='Median')

# Append the new rows to the DataFrame
percentage_change_df_fsi_final = pd.concat([percentage_change_df_fsi, aggregate_row.to_frame().T, median_row.to_frame().T])

percentage_change_df_fsi_final['Percentage Change in FSI Score'] = percentage_change_df_fsi_final['Percentage Change in FSI Score'].round(1)

# Display the final DataFrame
print(percentage_change_df_fsi_final)

                Percentage Change in FSI Score
Aruba                                     -3.3
Bahrain                                  -20.2
Bermuda                                  -11.9
Cayman Islands                             1.2
Mongolia                                  -2.7
Namibia                                  -14.1
Oman                                       0.1
Tunisia                                   -0.2
Average                                   -6.4
Median                                    -3.0


In [208]:
percentage_change_df_fdi_final.to_csv('../data/clean/percentage_change_df_fdi.csv', index=True)

In [ ]:
percentage_change_df_fsi_final.to_csv('../data/clean/percentage_change_df_fsi.csv', index=True)

: 